In [ ]:
from init import init_LLM_PURE, init_LLM_STORY, init_SDXL, init_Translator, init_TTM
from models import LLM_story, SDXL, TL, TTM
from utils import ordinal

In [ ]:
LLM_PURE_ = init_LLM_PURE()

In [ ]:
SDXL_ = init_SDXL

In [ ]:
language_dict = init_Translator

In [ ]:
TTM_ = init_TTM

In [ ]:
# def init_models():
#     LLM_PURE_ = init_LLM_PURE()
#     SDXL_ = init_SDXL
#     language_dict = init_Translator
#     TTM_ = init_TTM
#     
#     return LLM_PURE_, SDXL_, TTM_

In [ ]:
def start_gen(Params, LLM_PURE_): # after configuration page, while reading the prolog
    #update Params
    keys_to_update = ["title", "description"]
    for key in keys_to_update:
        if key in Params:
            Params[key] = TL(Params[key], from_code=Params["language"])
    
    LLM_STORY_ = init_LLM_STORY(LLM_PURE_, Params)
    
    LLM_STORY_, chapter, text = LLM_story(LLM_STORY_, "Write now only the 1st chapter!", mode="story") # type: ignore
    LLM_STORY_, decisions = LLM_story(LLM_STORY_, f"Create now {Params["num_decisions"]} decisions how the story could continue.", mode="decisions") # type: ignore
    
    image_path = SDXL(SDXL_, LLM_PURE_, text, chap=1, decision=1)
    sound_path = TTM(TTM_, LLM_PURE_, text, chap=1, decision=1)
    
    language = Params["language"]
    return LLM_STORY_, Params, TL(text, to_code=language), TL(chapter, to_code=language), TL(decisions, to_code=language), image_path, sound_path

In [ ]:
def next_gen(Params, LLM_STORY_, LLM_PURE_, chap_num, decision):
    LLM_STORY_, chapter, text = LLM_story(LLM_STORY_, f"Write now the {ordinal(chap_num)} chapter of {params["NUM_CHAPTERS"]} with using decision {decision}.", mode="story") # type: ignore
    LLM_STORY_, decisions = LLM_story(LLM_STORY_, f"Create now {Params["num_decisions"]} decisions how the story could continue.", mode="decisions") # type: ignore
    
    image_path = SDXL(SDXL_, LLM_PURE_, text, chap_num, decision)
    sound_path = TTM(TTM_, LLM_PURE_, text, chap_num, decision)
    
    language = Params["language"]
    return LLM_STORY_, TL(text, to_code=language), TL(chapter, to_code=language), TL(decisions, to_code=language), image_path, sound_path